## **<div style="text-align: center">ENCODER</div>**
=======================================================================================================================================

    + this is a sub-system that helps create embeddings of audio samples
    + this is trained on audio samples and is trained to minimize the similarty of embeddings of utterences that from the same speaker and maximize that between different speakers
=======================================================================================================================================

set the paths

In [13]:
dataset_root = Path("C:/Users/Swaroop/Downloads/train-clean-100/LibriSpeech/train-clean-100/train-clean-100/")
outdir = Path("./encoder/outdir")

### **SECTION 1:** Dataset preparation 

stpes of data preparation - 
1. load the audio files using `librosa` 
2. preprocess the wav and save it as a numpy file in the resptive speaker dir in the outdir.
3. store the meta data of every utterence in the speaker dir in a `sources.txt` with the same folder.
    
    `one such entry` :
         
        "495_26-495-0002.npy ,  C:\Users\Swaroop\Downloads\train-clean-100\LibriSpeech\train-clean-100\26\495\26-495-0002.flac"


In [1]:
from encoder import params_data, params_model
from encoder.audio import *

from pathlib import Path
from typing import Optional, Union
from warnings import warn
import warnings
import numpy as np
import librosa
warnings.filterwarnings("ignore")

NOTE: we are implementing the preprocessing tailored for libritts dataset as it the only datset we have  used in this project.

other functions pertaining to the preprocessing of other datasets have been omited.

preprocessing functions call heirarchy -  `preprocess_datset()` --**calls**--> `preprocess_speaker()` --**calls**--> `preprocess_utterences()`

In [6]:
from encoder.audio import * 
from encoder.params_data import * 

_AUDIO_EXTENSIONS = ("wav", "flac", "m4a", "mp3")

def _preprocess_speaker(speaker_dir: Path, datasets_root: Path, out_dir: Path, skip_existing: bool = True):
    # Give a name to the speaker that includes its dataset
    speaker_name = "_".join(speaker_dir.relative_to(datasets_root).parts)

    # Create an output directory with that name, as well as a txt file containing a
    # reference to each source file.
    speaker_out_dir = out_dir.joinpath(speaker_name)
    speaker_out_dir.mkdir(exist_ok=True)
    sources_fpath = speaker_out_dir.joinpath("_sources.txt")

    # There's a possibility that the preprocessing was interrupted earlier, check if
    # there already is a sources file.
    if sources_fpath.exists():
        try:
            with sources_fpath.open("r") as sources_file:
                existing_fnames = {line.split(",")[0] for line in sources_file}
        except:
            existing_fnames = {}
    else:
        existing_fnames = {}

    # Gather all audio files for that speaker recursively
    sources_file = sources_fpath.open("a" if skip_existing else "w")
    audio_durs = []
    for extension in _AUDIO_EXTENSIONS:
        for in_fpath in speaker_dir.glob("**/*.%s" % extension):
            # Check if the target output file already exists
            out_fname = "_".join(in_fpath.relative_to(speaker_dir).parts)
            out_fname = out_fname.replace(".%s" % extension, ".npy")
            if skip_existing and out_fname in existing_fnames:
                continue

            # Load and preprocess the waveform
            wav = preprocess_wav(in_fpath)
            if len(wav) == 0:
                continue

            frames = wav_to_mel_spectrogram(wav)
            if len(frames) < partials_n_frames:
                continue

            out_fpath = speaker_out_dir.joinpath(out_fname)
            np.save(out_fpath, frames)
            sources_file.write("%s,%s\n" % (out_fname, in_fpath))
            audio_durs.append(len(wav) / sampling_rate)

    sources_file.close()



test_run

In [19]:
dataset_root = Path("C:/Users/Swaroop/Downloads/train-clean-100/LibriSpeech/train-clean-100")

In [7]:
test_out_dir = Path(".").joinpath("test-Out_directory2")
test_out_dir.mkdir(exist_ok= True)
_preprocess_speaker(dataset_root.joinpath("19"), datasets_root= dataset_root, out_dir=test_out_dir)

In [ ]:
def apply_preprocessing_to_all_speakers(data_set_root: Path):
    speakers = list(data_set_root.glob("*"))
    for speaker in speakers:
        _preprocess_speaker(speaker, datasets_root= dataset_root, out_dir=outdir)

apply the preprocessing to all the speakers. (this could take a while)

In [ ]:
apply_preprocessing_to_all_speakers(dataset_root)

run the above function to apply preprocessing to all the data.

### **SECTION 2:** Model training

Note: the imput to train the model is the same as theoutdir that we generated during preprocessing

running the training function

In [ ]:
from pathlib import Path
from encoder.model2 import SpeakerEncoder
from encoder.train import train
import torch 
clean_dataset_root = Path("./outdir2/")
model = SpeakerEncoder(torch.device('cpu'), torch.device('cpu'))
new_models_dir = Path('./new_model_dir/')
new_models_dir.mkdir(exist_ok=True)
train("1", clean_dataset_root, new_models_dir, 10, 10, 10)


### **SECTION 3:** Inference

encoder inference module implementation 

In [9]:
def load_and_embed(encoder_model_path, audio_file_path):
    
    # imports    
    from encoder import inference2 as encoder
    from encoder.audio import preprocess_wav
    import librosa
    
    # model loading    
    encoder.load_model(encoder_model_path)    
    original_wav, sampling_rate = librosa.load(audio_file_path)
    preprocessed_wav = preprocess_wav(original_wav, sampling_rate)
    
    # generating embeds    
    embed = encoder.embed_utterance(preprocessed_wav, using_partials = False)
    
    return embed

### **SECTION 4:** COMPARING RESULTS

In [7]:
our_model = Path("./new_model_dir/1/encoder.pt")
saved_model = Path("C:/Users/Swaroop/OneDrive/Documents/Real-Time-Voice-Cloning-master/saved_models/default/encoder.pt")
audio_fpath = Path("C:/Users/Swaroop/Downloads/both.flac")

In [10]:
embed = load_and_embed(our_model, audio_fpath)
print(embed)

Loaded encoder "encoder.pt" trained to step 31


c:\Users\Swaroop\real-time-voice-cloner\encoder\audio.py:42: FutureWarning: Pass orig_sr=22050, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  wav = librosa.resample(wav, source_sr, sampling_rate)
c:\Users\Swaroop\real-time-voice-cloner\encoder\audio.py:58: FutureWarning: Pass y=[-0.02705792 -0.04254879 -0.03892137 ... -0.03959629 -0.04196231
 -0.04154412], sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  frames = librosa.feature.melspectrogram(


[0.00660599 0.13609818 0.08857831 0.0828993  0.02956464 0.03854567
 0.         0.19567405 0.02477263 0.01610441 0.06377555 0.
 0.         0.         0.05463692 0.15310887 0.01248206 0.15648209
 0.         0.         0.05015788 0.01215473 0.         0.
 0.09444315 0.         0.         0.10352036 0.0528067  0.12740873
 0.06806444 0.         0.09161198 0.         0.06622141 0.
 0.         0.         0.         0.02857352 0.         0.
 0.05949973 0.         0.01280187 0.         0.05263649 0.
 0.         0.06459756 0.03689168 0.         0.         0.10369769
 0.06628977 0.         0.         0.15247673 0.         0.
 0.02736049 0.         0.1837995  0.         0.01261522 0.08031336
 0.04983153 0.         0.         0.06262928 0.         0.
 0.06637316 0.14579009 0.0546518  0.05780488 0.         0.03637395
 0.04982163 0.02094671 0.         0.04942364 0.         0.
 0.         0.         0.         0.09269736 0.06819349 0.
 0.14063959 0.         0.         0.00898665 0.         0.12328354


In [11]:

embed = load_and_embed(saved_model, audio_fpath)
print(embed)

Loaded encoder "encoder.pt" trained to step 1564501
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.09819598e-01 0.00000000e+00 0.00000000e+00 1.43052042e-02
 0.00000000e+00 0.00000000e+00 9.58929583e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 3.67490016e-02
 1.62922353e-01 0.00000000e+00 5.86892851e-02 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.07819758e-01 0.00000000e+00 9.01186392e-02
 0.00000000e+00 0.00000000e+00 2.71619111e-01 2.09977657e-01
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.63994402e-01
 0.00000000e+00 0.00000000e+00 1.01646684e-01 4.18301821e-02
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.28442049e-01 0.00000000e+00 2.21534491e-01 0.00000000e+00
 1.72901466e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.